# Introducción a NumPy

### ¿Qué es NumPy?

Este notebook describe las técnicas para cargar, almacenar y manipular eficazmente datos en memoria en Python.
Los datos pueden provenir de una amplia variedad de fuentes y tener formatos muy diferentes, incluyendo colecciones de documentos, colecciones de imágenes, colecciones de clips de sonido, colecciones de medidas numéricas, o casi cualquier otra cosa. A pesar de esta aparente heterogeneidad, todos estos formatos de datos pueden expresarse como matrices de números.

Por ejemplo, las imágenes digitales pueden considerarse simplemente matrices bidimensionales de números que representan el brillo o color de los píxeles en cada posición. Los clips de sonido pueden considerarse matrices unidimensionales de intensidad en función del tiempo. El texto puede convertirse de varias maneras en representaciones numéricas, por ejemplo, dígitos binarios que cuantifican la frecuencia de ciertas palabras o pares de palabras.

Sean cuales sean los datos, el primer paso para poder analizarlos será transformarlos en matrices de números. Por esta razón, el almacenamiento y la manipulación eficiente de matrices numéricas es absolutamente fundamental para en la *ciencia de datos*. A continuación veremos las herramientas especializadas que tiene Python para manejar dichos arrays numéricos: el paquete **NumPy**, y el paquete **pandas**.

**NumPy** proporciona una implementación eficiente para almacenar y manipular arrays de datos densos. En cierto modo, los **arrays de NumPy** son como el tipo `list` de Python, pero los arrays de NumPy proporcionan un almacenamiento y unas operaciones mucho más eficientes a medida que aumenta el tamaño de los datos. Los arrays de NumPy forman el núcleo de casi todo el ecosistema de herramientas de ciencia de datos en Python. En general, NumPy proporciona estructuras de datos y rutinas básicas optimizadas para **manipular datos numéricos multidimensionales**. NumPy está implementado en gran medida en **código C pre-compilado**.  Su eficiencia en el procesamiento datos hace que NumPy sea utilizado internamente por muchas otras librerías de computación numérica de Python como `SciPy`, `matplotlib` o `pandas`.

In [14]:
import numpy
numpy.__version__

'1.24.4'

Lo habitual es simplificar el nombre del paquete a una abreviatura que ya se considera estándar.

In [15]:
import numpy as np

## Entendiendo los tipos de datos de Python

### Un entero en Python es más que un entero

La implementación estándar de Python está escrita en C.
Esto significa que cada objeto de Python es simplemente una estructura de C que contiene no sólo su valor sino también otra información. Por ejemplo, cuando definimos un entero en Python, como `x = 10000`, `x` no es sólo un entero "en bruto", sino que en realidad es un puntero a una estructura en C que contiene varios campos. Revisando el código fuente de Python, encontramos que la definición del tipo entero (largo), una vez que las macros C se expanden, es la siguiente:

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

Un único entero en Python contiene en realidad cuatro partes:

- `ob_refcnt`, un *contador* de referencias que ayuda a Python a gestionar automáticamente la asignación y liberación de memoria
- `ob_type`, que codifica el tipo de la variable
- `ob_size`, que especifica el tamaño de `ob_digit`
- `ob_digit`, que contiene el valor entero real que esperamos que represente la variable de Python.

Esto significa que hay una cierta sobrecarga en el almacenamiento de un entero en Python en comparación con un entero en un lenguaje compilado como C, como se ilustra en la figura:

![Integer Memory Layout](figures/cint_vs_pyint.png)

Aquí `PyObject_HEAD` es la parte de la estructura que contiene todos los campos de la estructura mencionados anteriormente a excepción de `ob_digit`.

Nótese la diferencia. Mientras que un entero en C es esencialmente una etiqueta para una posición en memoria cuyos bytes codifican un valor entero, un entero en Python es un puntero a una posición en memoria que contiene toda la información del objeto de Python, incluyendo los bytes que contienen el valor del entero. Esta información adicional en la estructura de enteros de Python es lo que permite escribir código en Python de forma tan flexible y dinámica. Sin embargo, toda esta información adicional en los tipos de Python tiene un coste, que se hace especialmente evidente en las estructuras que combinan muchos de estos objetos.

### Una lista de Python es más que una lista

Consideremos ahora lo que sucede cuando usamos una estructura de datos de Python que contiene muchos objetos de Python. El contenedor multi-elemento mutable estándar en Python es la lista. Podemos crear una lista de enteros de la siguiente manera:

In [16]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
type(l[0])

int

Recordemos que, debido al tipado dinámico de Python, también podemos definir una lista de varios tipos diferentes.

In [18]:
l = [True, "2", 3.0, 4]
[type(item) for item in l]

[bool, str, float, int]

Pero esta flexibilidad tiene un precio: para permitir estos tipos flexibles, cada elemento de la lista debe contener su propia información de tipo, número de referencias y otra información adicional, es decir, cada elemento es un objeto Python completo. En el caso de que todas las variables sean del mismo tipo es mucho más eficiente almacenar los datos en un array de dicho tipo fijo.
La diferencia entre una lista de tipo dinámico y una matriz de tipo fijo (estilo NumPy) se ilustra en la figura:

![Array Memory Layout](figures/array_vs_list.png)

El array contiene esencialmente un único puntero a un bloque contiguo de datos. Por otro lado, la lista de Python contiene un puntero a un bloque de punteros, cada uno de los cuales apunta a su vez a un objeto completo de Python como el entero de Python que vimos antes. De nuevo, la ventaja de la lista es la flexibilidad. Como cada elemento de la lista es una estructura completa que contiene tanto datos como información de tipo, la lista puede llenarse con datos de cualquier tipo deseado.
Las matrices de tipo fijo al estilo de NumPy carecen de esta flexibilidad, pero son mucho más eficientes para almacenar y manipular datos.

## Los arrays de NumPy
En primer lugar, podemos utilizar ``np.array`` para crear arrays a partir de listas de Python:

In [19]:
np.array([1, 4, 2, 5, 3])  # array de enteros

array([1, 4, 2, 5, 3])

In [20]:
np.array(['a', 'b', 'c', 'd', 'e', 'f'])

array(['a', 'b', 'c', 'd', 'e', 'f'], dtype='<U1')

Sin embargo, ahora no podemos mezclar tipos. Si lo hacemos, Numpy intentará usar un tipo suficientemente general como para incluir a todos los elementos. En este caso al poner un valor real, todo pasa a ser de tipo *float*.

In [21]:
np.array([3.14, 4, 2, 3])  # array de reales

array([3.14, 4.  , 2.  , 3.  ])

Los arrays de NumPy contienen **valores de un solo tipo** (enteros, reales o complejos de distinta precisión). Podemos indicar explícitamente el tipo del array con el argumento ``dtype``:

In [22]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

A diferencia de las listas de Python, los arrays de NumPy son de **tamaño fijo**, definido en el momento de su creación.

Además, los arrays de NumPy pueden ser explícitamente *multidimensionales*, al contrario que las listas (unidimensionales).

He aquí una forma de inicializar un array multidimensional usando una lista de listas:

In [23]:
# listas anidadas generan arrays multicimensionales
x = np.array([[2, 3, 4], [4, 5, 6], [6, 7, 8]])
print(x)

[[2 3 4]
 [4 5 6]
 [6 7 8]]


Las listas se tratan como filas de la matriz bidimensional resultante.

In [24]:
print(x[0])  # primera fila
print(x[-1])  # última fila

[2 3 4]
[6 7 8]


También es posible acceder a la matriz bidimensional por columnas:

In [25]:
print(x[:, 0])  # primera columna
print(x[:, -1])  # última columna

[2 4 6]
[4 6 8]


Los intentos de acceder a un elementos que no existen generan una excepción IndexError:

In [26]:
x[0, 4]  # Genera una excepción IndexError

IndexError: index 4 is out of bounds for axis 1 with size 3

Como ya hemos visto, en un array multidimensional se puede acceder a los elementos utilizando una tupla de índices separada por comas:

In [28]:
print(x[2, 1])

7


Los valores pueden ser modificados con la misma notación:

In [29]:
x[0, 0] = 20
x

array([[20,  3,  4],
       [ 4,  5,  6],
       [ 6,  7,  8]])

Cada array tiene los atributos `ndim` (el número de dimensiones), `shape` (el tamaño de cada dimensión), `size` (el tamaño total del array) y `dtype` (el tipo de datos de los elementos):

In [45]:
print(f"ndim: {x.ndim}")
print(f"shape: {x.shape}") # tupla con las dimensiones del array
print(f"size: {x.size}")
print(f"dtype: {x.dtype}")

ndim: 2
shape: (3, 3)
size: 9
dtype: int64


Otros atributos interesantes son `itemsize`, que indica el tamaño (en bytes) de cada elemento del array, y `nbytes`, que indica el tamaño total (en bytes) del array:

In [46]:
print(f"itemsize: {x.itemsize} bytes")
print(f"nbytes: {x.nbytes} bytes")

itemsize: 8 bytes
nbytes: 72 bytes


En general, esperamos que ``nbytes`` sea igual a ``itemsize`` por ``size``.

Nótese que, a diferencia de las listas de Python, los arrays de NumPy tienen un tipo fijo. Esto significa, por ejemplo, que se intenta insertar un valor de punto flotante en un array de enteros, el valor se truncará.

In [32]:
x[0, 0] = 3.14159  # Este valor se truncará a 3
x

array([[3, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

También podemos acceder a subarrays con la notación *slice* que conocemos de las listas Python(`:`). Para acceder a una parte de una matriz `x`:

``` python
x[start:stop:step]
```

In [33]:
y = np.array([x for x in range(10,20)])
y

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [34]:
y[:2]  # primeros 2 elementos

array([10, 11])

In [35]:
y[2:]  # elementos después del índice 2

array([12, 13, 14, 15, 16, 17, 18, 19])

In [36]:
y[2:5]  # subarray entre índices 2 y 5

array([12, 13, 14])

**IMPORTANTE**: Los *slices* de arrays devuelven **vistas** de los datos del array, al contrario que en las listas de Python, donde los slices generan **copias**. Esto permite acceder y procesar fragmentos de arrays NumPy sin necesidad de copiar el array de datos original.

In [37]:
y_sub = y[2:5]
y_sub[0] = 0
print(y)

[10 11  0 13 14 15 16 17 18 19]


In [38]:
l = [x for x in range(10,20)]
print(l)
l_sub = l[2:5]
l_sub[0] = 0
print(l)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


## Operaciones con arrays de Numpy: Universal Functions (UFuncs)

NumPy es importante en el mundo de la ciencia de datos de Python porque proporciona una interfaz fácil y flexible para la computación optimizada con arrays de valores. La clave para hacer que la computación con arrays de NumPy sea rápida es usar operaciones *vectorizadas*, generalmente implementadas a través de las *funciones universales* de NumPy (*ufuncs*) que pueden usarse para hacer cálculos repetidos sobre elementos del array de una forma mucho más eficiente.

In [39]:
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
print(values)
compute_reciprocals(values)

[6 1 4 4 8]


array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

Esta implementación probablemente resulte bastante natural para alguien que tenga, por ejemplo, conocimientos de C o Java. Pero si medimos el tiempo de ejecución de este código para una entrada grande, vemos que esta operación es muy lenta; sorprendentemente lenta. Vamos a comparar esto con la orden *mágica* `%timeit` del notebook, que nos permite saber lo que tarda en ejecutarse la orden que la sigue.

In [48]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1.37 s ± 356 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


El tiempo empleado en el cálculo de 10 millones de operaciones puede variar dependiendo del entorno en que se ejecute; en el mejor de los casos, supera ampliamente el segundo de duración. Cuando incluso los teléfonos móviles tienen velocidades de procesamiento que se miden en Giga-FLOPS (miles de millones de operaciones por segundo), vemos que resulta absurdamente lento. El cuello de botella aquí no son las operaciones en sí, sino la comprobación dinámica de tipos y las indirecciones que Python debe hacer en cada ciclo del bucle. Cada vez que se calcula el recíproco, Python examina primero el tipo del objeto y hace una búsqueda dinámica de la función correcta a utilizar para ese tipo. Si estuviéramos trabajando en código compilado, esta especificación de tipo sería conocida antes de que el código se ejecute y el resultado podría ser calculado mucho más eficientemente. Todo eso sin olvidar que los arrays NumPy almacenan los elementos en memoria densamente empaquetados, por lo que consigue los beneficios en términos de rendimiento derivados del principio de localidad de referencia.

Para muchos tipos de operaciones, NumPy proporciona un interfaz a una versión de cada rutina ya compilada para cada tipo de datos para hacerla sumamente eficiente. Esto se conoce como una **operación *vectorizada***. Esto puede lograrse simplemente operando directamente con el array, sabiendo que esa operación se aplicará a cada elemento por separado.
Este enfoque vectorizado está diseñado para llevar el bucle hasta la capa compilada que tiene por debajo NumPy, lo que lleva a una ejecución mucho más rápida.

**NOTA**: En el contexto de lenguajes de programación de alto nivel como Python, Matlab o R, el término **vectorizado** hace referencia al uso de código pre-compilado optimizado, escrito en un lenguaje de más bajo nivel (por ejemplo, C) con el fin de realizar operaciones matemáticas sobre una secuencia de datos, sustituyendo las iteraciones explícitas escritas en el código del lenguaje nativo (e.g., un bucle for en Python).

Para ilustrar la diferencia, comparemos los resultados de las dos operaciones siguientes y su tiempo de ejecución:

In [49]:
print(compute_reciprocals(values))
print(1.0 / values) # true_divide ufunc in NumPy

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


Si observamos el tiempo de ejecución de nuestro gran array, vemos que se completa órdenes de magnitud más rápido que el bucle de Python:

In [50]:
%timeit (1.0 / big_array)

1.14 ms ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Las operaciones *vectorizadas* en NumPy se implementan a través de *ufuncs*, cuyo propósito principal es ejecutar rápidamente operaciones repetidas sobre valores en arrays de NumPy.
Las *ufuncs* son extremadamente flexibles. Antes vimos una operación entre un escalar y un array, pero también podemos operar entre dos arrays:

In [51]:
a = np.arange(5)
print(a)
print(a * 5)  # multiplica cada elemento por 5

b = np.arange(1, 6)
print(b)
print(a/b)  # divide elemento a elemento los dos arrays

[0 1 2 3 4]
[ 0  5 10 15 20]
[1 2 3 4 5]
[0.         0.5        0.66666667 0.75       0.8       ]


## Referencias

* [Numpy](https://numpy.org/)
  * [Numpy · Quickstart](https://numpy.org/doc/stable/user/quickstart.html)
  * [Numpy · Basics for Beginners](https://numpy.org/doc/stable/user/absolute_beginners.html)
  * [Numpy · API Reference](https://numpy.org/doc/stable/reference/index.html)